In [6]:
import pandas as pd
import json
import requests
import numpy as np
import openpyxl
import xlsxwriter
from datetime import datetime

# DRIVER CODE, GO BELOW FOR ACTUAL USE

In [7]:
def generate_quantity(protein_list):
    """helper function for retrieve. generates protein quantities from a given list of samples.
    returns a tuple with (total quantity of protein, lot number dictionary with individual quantities)"""
    
    protein_quantity = 0
    LN_dict = {}
    
    # iterate through the given list and generate quantities of the material sample by sample. 
    # Since custom fields did not have (mL) or mg/mL specified, we must parse out alphabet characters
    # while retaining numerals and the '.' symbol
    for sample in protein_list:
        quantity = sample.get('quantity')
        # sometimes people don't fill in the boxes all the way >:(
        if quantity == None:
            continue
        # parse quantity to only get numerals, then convert to float
        quant_float = ''
        for char in quantity:
            quant_char = char
            if quant_char.isdigit() == True or quant_char == '.':
                quant_float = quant_float + quant_char
        quant_float = float(quant_float)
        # concentration is a custom value, meaning it appears in the meta section.
        # after correct sampletypemetaIDs are found, perform the same parsing as above
        meta = (sample.get('meta'))
        conc_float = ''
        for meta_type in meta:
            # search through meta to find the sampleTypeMetaID corresponding to concentration
            if meta_type.get('sampleTypeMetaID') == 188352 or meta_type.get('sampleTypeMetaID') == 188337:
                concentration = meta_type.get('value')
            # also, capture the lot number
            if meta_type.get('sampleTypeMetaID') == 188283 or meta_type.get('sampleTypeMetaID') == 188349:
                lot_number = meta_type.get('value')
        for char in concentration:
            conc_char = char
            if conc_char.isdigit() == True or conc_char == '.':
                conc_float = conc_float + conc_char
        # sometimes people don't fill in the boxes all the way >:(
        if conc_float == '':
            continue
        else:
            conc_float = float(conc_float)
            sample_quantity = (conc_float*quant_float)
            # add the sample quantity to total lot quantity in dictionary
            if lot_number in LN_dict:
                LN_dict[lot_number] = LN_dict[lot_number] + sample_quantity 
            else:
                LN_dict[lot_number] = sample_quantity
            protein_quantity = sample_quantity + protein_quantity
    return (protein_quantity, LN_dict)     

def retrieve(freezer_id):
    """This function submits GET requests to retrieve samples of a certain PN from a given freezer.
    Since there are two main sample types here, Antibody, and Antigen, there are thus two
    sampleTypeMetaID's for any given variable. For example, the 'Part Number' custom variable actually
    has two sampleTypeMetaID's (one for antibody, the other for antigen), 191796 or 191790. """
    
    # authentication token retrieval, stored in 'token'
    data = {
      "username": "###",
      "password": "###"
    }
    response = requests.post('https://us.elabjournal.com/api/v1/auth/user', data=data)
    response = response.json()
    token = response.get('token')
    # use the retrieved authentication token in the header of all requests
    headers = {
            'Accept': 'application/json',
            'X-Requested-With': 'Swagger',
            'Authorization': token
    }
    
    # set up to iterate through storage unit page by page (otherwise throws error that data is too large)
    # api call is set up to include quantity and page number (for iterative purposes, as shown above)
    # There is currently no way to search a compartment through the API without restricting access
    # to the custom fields.
    url_first_half = 'https://us.elabjournal.com/api/v1/samples?$expand=meta&$expand=quantity&$page='
    url_second_half = '&storageID=' + str(freezer_id)
    loop = True
    page_number = 0
    samples = []
    while loop == True:
        url = url_first_half + str(page_number) + url_second_half
        # actual request is performed below
        page_of_samples = requests.get(url, headers=headers)
        page_of_samples = page_of_samples.json()
        print('On Page', page_number)
        # manually break out once we are retrieving empty pages
        if page_of_samples.get('recordCount') == 0:
            loop = False
            break
        # grabs page as a list
        page_of_samples = page_of_samples.get('data')
        page_number += 1
        samples = samples + page_of_samples
        
    # create lists of antibodies (604.6J9) and antigens (TRX-XBC-F6 Hinge Dimer)
    antibody = []
    antigen = []
    for sample in samples:
        part_number = 'XX00XX' #ensure part_number gets reset each loop, otherwise things randomly get added
        meta = (sample.get('meta'))
        for custom in meta:
            if custom.get('sampleTypeMetaID') == 191796 or custom.get('sampleTypeMetaID') == 191790:
                part_number = custom.get('value')
        if part_number == '301496':
            antibody.append(sample)
        elif part_number == '301492':
            antigen.append(sample)
    
    # sorting / displaying data
    samples = pd.DataFrame(samples)
    samples.to_excel('fullsamples.xlsx')
    antibody_pd = pd.DataFrame(antibody)
    antibody_pd.to_excel('amount.xlsx')
    antibody_quantity = generate_quantity(antibody)
    antigen_quantity = generate_quantity(antigen)
    antibody_by_lot = pd.DataFrame(antibody_quantity[1], index=[0])
    antigen_by_lot = pd.DataFrame(antigen_quantity[1], index=[0])
    print('------------------------------------------------')
    print('Total amount of 6J9 (mg):', antibody_quantity[0])
    print('Lots of 6J9:', antibody_by_lot.T) # need to transpose, or displays horizontally lol
    print('------------------------------------------------')
    print('Total amount of F6 (mg):', antigen_quantity[0])
    print('Lots of F6:', antigen_by_lot.T)


# RUN BELOW TO CHECK MFG FREEZER 148

In [8]:
freezer_id = 69875
retrieve(freezer_id)

On Page 0
On Page 1
On Page 2
------------------------------------------------
Total amount of 6J9 (mg): 66536.95180000005
Lots of 6J9:                   0
AJ6185      28.7500
AJ6187      25.8000
AL6152      92.6400
AL6192      38.6400
BA6418     182.8500
BA6417     197.6850
AJ6218       5.5900
AJ6324      51.0300
AJ6352      36.2600
BA6047      50.8500
BA6048      51.4800
BA6167      47.7900
AE6097      22.0000
AE6096      11.2600
AE6115       5.9000
AE6277      20.9600
AJ6290     739.6000
DFT7375   7624.7600
BA6170      33.6000
BA6168      29.3400
BA6169      45.3600
DFT6951     13.7000
DFT7343   6416.8300
DFT7047   3080.0900
DFT7023   5505.2800
AE6166     122.1000
CJ6079    5050.0000
DFT6656    716.5900
DFT6661A   349.7760
DFT6661B   352.2960
DFT6944   8776.9965
DFT6847   3743.0000
DFT6872   1140.7500
DFT6997   4249.4683
DFT7281   6794.0000
DFT6002     47.8500
DFT8565   4190.0000
CH6246    6646.0800
------------------------------------------------
Total amount of F6 (mg): 4433.60999

# RUN BELOW TO CHECK ALL

In [5]:
freezer_id = ''
retrieve(freezer_id)

On Page 0
On Page 1
On Page 2
On Page 3
On Page 4
On Page 5
On Page 6
On Page 7
On Page 8
On Page 9
On Page 10
On Page 11
On Page 12
On Page 13
On Page 14
On Page 15
On Page 16
On Page 17
On Page 18
On Page 19
On Page 20
On Page 21
On Page 22
On Page 23
On Page 24
On Page 25
On Page 26
On Page 27
On Page 28
On Page 29
On Page 30
On Page 31
On Page 32
On Page 33
On Page 34
On Page 35
On Page 36
On Page 37
On Page 38
On Page 39
On Page 40
On Page 41
On Page 42
On Page 43
On Page 44
On Page 45
On Page 46
On Page 47
On Page 48
On Page 49
On Page 50
On Page 51
On Page 52
On Page 53
On Page 54
On Page 55
On Page 56
On Page 57
------------------------------------------------
Total amount of 6J9 (mg): 54952.76720000001
Lots of 6J9:                   0
DFT6587B    27.1500
DFT6804     90.6000
AL6192      38.6400
AE6277      20.9600
DFT6661B   384.9160
DFT6847   7019.8000
DFT6448     45.6000
AJ6185      28.7500
AJ6187      25.8000
AJ6290     739.6000
DFT6566     29.5800
BA6168      29.3400
DFT628